In [2]:
import numpy as np
import pandas as pd

## read training data

In [3]:
#read txt file
def read_txt(filename):
    with open(filename, 'r') as f:
        data = f.readlines()
    return data


traininglist1 = read_txt('../BERT/data_processed/train(no_BIO).txt')
#traininglist2 = read_txt('./trainingdata2_noPHI.txt')
#traininglist = traininglist1 + traininglist2
#traininglist3 = read_txt('./train_withoutphi.txt')

#you could pick one of the trainingset to train the model. 
#Traininglist1 is the first trainingset, traininglist2 is the second trainingset, traininglist3 is the combination.
traininglist = traininglist1

#print(list)
for i in range(len(traininglist)):
    traininglist[i] = traininglist[i].split()
#print(list)

trainingdataframe = pd.DataFrame(traininglist, columns=['text', 'label'])
print(trainingdataframe.shape)
trainingdataframe.head()

df_training = trainingdataframe.fillna(method='ffill')
df_training.text.nunique(), df_training.label.nunique()  #check the number of unique words and labels

(489694, 2)


(53284, 9)

In [4]:
df_training.groupby('label').size().reset_index(name='counts') #check the number of each label

,label,counts
0,CAD,5620
1,DIABETES,2284
2,FAMILY_HIST,254
3,HYPERLIPIDEMIA,590
4,HYPERTENSION,2188
5,MEDICATION,6090
6,O,470029
7,OBESE,292
8,SMOKER,2347


In [41]:
#delete the rows with label 'O', and generate a new dataframe without 'O' for training without 'O'
df_training.loc[df_training.label != 'CAD',"label"] = 'O'
df_training.groupby('label').size().reset_index(name='counts')


,label,counts
0,CAD,3873
1,O,333009


## read testing data

In [5]:
#read txt file
testinglist = read_txt('../BERT/data_processed/test(no_BIO).txt')
#print(list)

for i in range(len(testinglist)):
    testinglist[i] = testinglist[i].split()
#print(list)

testingdataframe = pd.DataFrame(testinglist,columns=['text','label'])
print(testingdataframe.shape)
testingdataframe.head()

(316727, 2)


,text,label
0,Record,SMOKER
1,date:,O
2,2069-04-07,O
3,Mr.,O
4,Villegas,O


In [6]:
df_testing = testingdataframe.fillna(method='ffill')
df_testing.text.nunique(), df_testing.label.nunique() #check the number of unique words and labels

(40140, 9)

In [7]:
df_testing.groupby('label').size().reset_index(name='counts') #check the number of each label

,label,counts
0,CAD,3951
1,DIABETES,1676
2,FAMILY_HIST,214
3,HYPERLIPIDEMIA,493
4,HYPERTENSION,1263
5,MEDICATION,4148
6,O,303157
7,OBESE,168
8,SMOKER,1657


In [8]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import Perceptron
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score

# Training with o class———choose one of the following training and testing method to run

### Vectorize the text samples into a 2D integer tensor and split the trainingdata into a training set and a validation set for primary results

In [114]:
trainingtext_with_o = df_training.drop('label',axis=1)  #drop the label column
traininglabel_with_o = df_training.label.values #get the label column
#print (trainingtext_with_o)
#print(traininglabel_with_o)

v = DictVectorizer(sparse=True) #sparse=True means the output is a sparse matrix
trainingtext_with_o = v.fit_transform(trainingtext_with_o.to_dict('records')) #to_dict('records') means the output is a list of dictionaries
#print(trainingtext_with_o)

classes = np.unique(traininglabel_with_o) #get the unique labels as classes for performance evaluation
classes = classes.tolist() #convert the numpy array to list
#print(classes)

x_train, x_test, y_train, y_test = train_test_split(trainingtext_with_o, traininglabel_with_o, test_size = 0.3, random_state=0) 

x_train.shape, y_train.shape,x_test.shape, y_test.shape

((235817, 42418), (235817,), (101065, 42418), (101065,))

In [26]:
x_train = df_training.drop('label',axis=1)  #drop the label column
y_train = df_training.label.values #get the label column
x_test = df_testing.drop('label',axis=1)  #drop the label column
y_test = df_testing.label.values #get the label column



In [27]:

tt = x_train.append(x_test)
ttt =  v.fit_transform(tt.to_dict('records'))

ttt.shape


C:\Users\Leste\AppData\Local\Temp\ipykernel_1212\4215444867.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tt = x_train.append(x_test)


(806421, 73054)

In [28]:
x_train.shape

(489694, 1)

In [29]:

x_train = ttt[:x_train.shape[0]]
x_test = ttt[x_train.shape[0]:]

In [115]:
from sklearn.utils import resample

# make two dataframes, each with only one class 
majority_df = x_train[y_train == "O"]
minority_df = x_train[y_train != "O"]

minority_df.shape, majority_df.shape
type(majority_df)
y_train

array(['O', 'O', 'O', ..., 'O', 'O', 'O'], dtype=object)

In [10]:
import numpy as np
from scipy.sparse import vstack
import random
random.seed(72)
row = np.random.permutation(majority_df.shape[0])[:2*minority_df.shape[0]]
t_lab = y_train[y_train == "CAD"]
d_majority_df = majority_df[row]
x_train = vstack((minority_df,d_majority_df))
y_train = np.concatenate((t_lab,y_train[row]))

x_train.shape, y_train.shape,x_test.shape, y_test.shape




NameError: name 'majority_df' is not defined

### Vectorize the text samples into a 2D integer tensor and use different datasets as training set and test set

In [ ]:
#use df_training as training set and df_testing as test set
trainingtext_with_o = df_training.drop('label',axis=1)
traininglabel_with_o = df_training.label.values
#print (trainingtext_with_o)
#print(traininglabel_with_o)

v = DictVectorizer(sparse=True)
x_train = v.fit_transform(trainingtext_with_o.to_dict('records'))
#print(trainingtext_with_o)
y_train = traininglabel_with_o

classes = np.unique(traininglabel_with_o)
classes = classes.tolist()
#print(classes)

testingtext_with_o = df_testing.drop('label',axis=1)
#print(testingtext_with_o)
x_test = v.transform(testingtext_with_o.to_dict('records'))
#print(x_test)
y_test = df_testing.label.values
#print(y_test)

x_train.shape, y_train.shape,x_test.shape, y_test.shape


In [31]:
new_classes = classes.copy()
#new_classes.pop() #remove the last element 'O' from the list
new_classes

array(['CAD', 'DIABETES', 'FAMILY_HIST', 'HYPERLIPIDEMIA', 'HYPERTENSION',
       'MEDICATION', 'O', 'OBESE', 'SMOKER'], dtype=object)

### Below you can train in different classifiers

### Perceptron

In [39]:
per = Perceptron(verbose=10, n_jobs=-1, max_iter=5) #this is the perceptron model
per.partial_fit(x_train, y_train,classes=classes)
y_pred=per.predict(x_test)
print(y_pred)
y_pred.to_csv('../BERT/per.txt',index=False)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 16 concurrent workers.


-- Epoch 1-- Epoch 1

-- Epoch 1
-- Epoch 1
-- Epoch 1
-- Epoch 1
-- Epoch 1
-- Epoch 1
-- Epoch 1
Norm: 31.29, NNZs: 979, Bias: -0.150000, T: 489694, Avg. loss: 0.008481
Total training time: 0.28 seconds.
Norm: 12.57, NNZs: 158, Bias: -0.080000, T: 489694, Avg. loss: 0.000502
Total training time: 0.37 seconds.
Norm: 19.87, NNZs: 395, Bias: -0.070000, T: 489694, Avg. loss: 0.003240
Total training time: 0.29 seconds.
Norm: 67.58, NNZs: 4567, Bias: 0.010000, T: 489694, Avg. loss: 0.022582
Total training time: 0.31 seconds.
Norm: 7.07, NNZs: 50, Bias: -0.080000, T: 489694, Avg. loss: 0.000449
Total training time: 0.31 seconds.
Norm: 9.17, NNZs: 84, Bias: -0.040000, T: 489694, Avg. loss: 0.000246
Total training time: 0.32 seconds.
Norm: 30.66, NNZs: 940, Bias: -0.020000, T: 489694, Avg. loss: 0.002235
Total training time: 0.34 seconds.
Norm: 37.46, NNZs: 1403, Bias: -0.010000, T: 489694, Avg. loss: 0.005298
Total training time: 0.34 seconds.
Norm: 24.76, NNZs: 613, Bias: -0.050000, T: 4896

[Parallel(n_jobs=-1)]: Done   2 out of   9 | elapsed:    0.4s remaining:    1.7s
[Parallel(n_jobs=-1)]: Done   3 out of   9 | elapsed:    0.4s remaining:    1.0s
[Parallel(n_jobs=-1)]: Done   4 out of   9 | elapsed:    0.4s remaining:    0.6s
[Parallel(n_jobs=-1)]: Done   5 out of   9 | elapsed:    0.4s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done   6 out of   9 | elapsed:    0.4s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done   7 out of   9 | elapsed:    0.5s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done   9 out of   9 | elapsed:    0.5s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   9 out of   9 | elapsed:    0.5s finished


['O' 'O' 'O' ... 'O' 'O' 'O']


AttributeError: 'numpy.ndarray' object has no attribute 'to_csv'

In [41]:
with open('../BERT/pred.txt', 'w') as f:
    for i in range(len(y_pred)):
        f.write(df_testing['text'][i] + ' ' + y_test[i] + ' ' + y_pred[i] + '\n')

### Support Vector Machine

In [ ]:
#support vector machine
svm = SGDClassifier(alpha=.00001, max_iter=100,penalty="elasticnet")
svm.partial_fit(x_train, y_train,classes=new_classes)

print (classification_report(y_pred=svm.predict(x_test), y_true=y_test, labels=new_classes))
#labels=classes means the performance evaluation is based on all the labels 
#labels=new_classes means the performance evaluation is based on all the labels except 'O'

### Naive Bayes

In [ ]:
#naive bayes
nb = MultinomialNB(alpha=.0005)
nb.partial_fit(x_train, y_train,classes=classes)

print (classification_report(y_pred=nb.predict(x_test), y_true=y_test, labels=new_classes))
#labels=classes means the performance evaluation is based on all the labels 
#labels=new_classes means the performance evaluation is based on all the labels except 'O'

In [ ]:
#random forest
rf = RandomForestClassifier(n_estimators=100, max_depth=20,random_state=553)
rf.fit(x_train, y_train)

print (classification_report(y_pred=rf.predict(x_test), y_true=y_test, labels=classes))
#labels=classes means the performance evaluation is based on all the labels 
#labels=new_classes means the performance evaluation is based on all the labels except 'O'

In [ ]:
#logistic regression
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(x_train, y_train)

print (classification_report(y_pred=lr.predict(x_test), y_true=y_test, labels=new_classes))
#labels=classes means the performance evaluation is based on all the labels 
#labels=new_classes means the performance evaluation is based on all the labels except 'O'

# Training without o———choose one of the following training and testing method to run

### Vectorize the text samples into a 2D integer tensor and split the trainingdata into a training set and a validation set for primary results

In [ ]:
trainingtext_without_o = df_training_withouto.drop('label',axis=1) #drop the label column
#print (trainingtext_without_o)

v = DictVectorizer(sparse=True)
trainingtext_without_o = v.fit_transform(trainingtext_without_o.to_dict('records')) 
#print(trainingtext_without_o)
traininglabel_without_o = df_training_withouto.label.values

classes_without_o = np.unique(traininglabel_without_o)
classes_without_o = classes_without_o.tolist()

x_without_o_train, x_without_o_test, y_without_o_train, y_without_o_test = train_test_split(trainingtext_without_o, traininglabel_without_o, test_size = 0.1, random_state=0)
x_without_o_train.shape, y_without_o_train.shape,x_without_o_test.shape, y_without_o_test.shape


### Vectorize the text samples into a 2D integer tensor and use different datasets as training set and test set

In [ ]:

trainingtext_without_o = df_training_withouto.drop('label',axis=1)
traininglabel_without_o = df_training_withouto.label.values

v = DictVectorizer(sparse=True)
x_without_o_train = v.fit_transform(trainingtext_without_o.to_dict('records'))
y_without_o_train = traininglabel_without_o

classes_without_o = np.unique(traininglabel_without_o)
classes_without_o = classes_without_o.tolist()


testingtext_without_o = df_testing.drop('label',axis=1)

x_without_o_test = v.transform(testingtext_without_o.to_dict('records'))
y_without_o_test = df_testing.label.values


x_without_o_train.shape, y_without_o_train.shape,x_without_o_test.shape, y_without_o_test.shape


### Below you can train in different classifiers

In [ ]:
#perceptron
per_no = Perceptron(verbose=10, n_jobs=-1, max_iter=20) #this is the perceptron model
per_no.partial_fit(x_without_o_train, y_without_o_train,classes=classes_without_o)

print (classification_report(y_pred=per_no.predict(x_without_o_test), y_true=y_without_o_test, labels=classes_without_o))


In [ ]:
#support vector machine
svm_no = SGDClassifier(alpha=.00001, max_iter=100,penalty="elasticnet")
svm_no.partial_fit(x_without_o_train, y_without_o_train,classes=classes_without_o)

print (classification_report(y_pred=svm_no.predict(x_without_o_test), y_true=y_without_o_test, labels=classes_without_o))

print(precision_score(y_pred=svm_no.predict(x_without_o_test), y_true=y_without_o_test, labels=classes_without_o, average='micro'))
print(recall_score(y_pred=svm_no.predict(x_without_o_test), y_true=y_without_o_test, labels=classes_without_o, average='micro'))
print(f1_score(y_pred=svm_no.predict(x_without_o_test), y_true=y_without_o_test, labels=classes_without_o, average='micro'))
print(precision_score(y_pred=svm_no.predict(x_without_o_test), y_true=y_without_o_test, labels=classes_without_o, average='macro'))
print(recall_score(y_pred=svm_no.predict(x_without_o_test), y_true=y_without_o_test, labels=classes_without_o, average='macro'))
print(f1_score(y_pred=svm_no.predict(x_without_o_test), y_true=y_without_o_test, labels=classes_without_o, average='macro'))
print(precision_score(y_pred=svm_no.predict(x_without_o_test), y_true=y_without_o_test, labels=classes_without_o, average='weighted'))
print(recall_score(y_pred=svm_no.predict(x_without_o_test), y_true=y_without_o_test, labels=classes_without_o, average='weighted'))
print(f1_score(y_pred=svm_no.predict(x_without_o_test), y_true=y_without_o_test, labels=classes_without_o, average='weighted'))

In [ ]:
#naive bayes
nb_no = MultinomialNB(alpha=0.01)
nb_no.partial_fit(x_without_o_train, y_without_o_train,classes=classes_without_o)

#打印分类报告，并保留四位小数
print (classification_report(y_pred=nb_no.predict(x_without_o_test), y_true=y_without_o_test, labels=classes_without_o))

print(precision_score(y_pred=nb_no.predict(x_without_o_test), y_true=y_without_o_test, labels=classes_without_o, average='micro'))
print(recall_score(y_pred=nb_no.predict(x_without_o_test), y_true=y_without_o_test, labels=classes_without_o, average='micro'))
print(f1_score(y_pred=nb_no.predict(x_without_o_test), y_true=y_without_o_test, labels=classes_without_o, average='micro'))
print(precision_score(y_pred=nb_no.predict(x_without_o_test), y_true=y_without_o_test, labels=classes_without_o, average='macro'))
print(recall_score(y_pred=nb_no.predict(x_without_o_test), y_true=y_without_o_test, labels=classes_without_o, average='macro'))
print(f1_score(y_pred=nb_no.predict(x_without_o_test), y_true=y_without_o_test, labels=classes_without_o, average='macro'))
print(precision_score(y_pred=nb_no.predict(x_without_o_test), y_true=y_without_o_test, labels=classes_without_o, average='weighted'))
print(recall_score(y_pred=nb_no.predict(x_without_o_test), y_true=y_without_o_test, labels=classes_without_o, average='weighted'))
print(f1_score(y_pred=nb_no.predict(x_without_o_test), y_true=y_without_o_test, labels=classes_without_o, average='weighted'))


In [ ]:
#random forest
rf = RandomForestClassifier(n_estimators=100, max_depth=30,random_state=533)
rf.fit(x_without_o_train, y_without_o_train)

print (classification_report(y_pred=rf.predict(x_without_o_test), y_true=y_without_o_test, labels=classes_without_o))



In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(x_without_o_train, y_without_o_train)

print (classification_report(y_pred=lr.predict(x_without_o_test), y_true=y_without_o_test, labels=classes_without_o))

In [ ]:
#decision tree
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier()
dt.fit(x_without_o_train, y_without_o_train)

print (classification_report(y_pred=dt.predict(x_without_o_test), y_true=y_without_o_test, labels=classes_without_o))

In [ ]:
#knn
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
knn.fit(x_without_o_train, y_without_o_train)

print (classification_report(y_pred=knn.predict(x_without_o_test), y_true=y_without_o_test, labels=classes_without_o))

# stanford NER tagger result calculation
## just for calculating the accuracy of stanford NER tagger, not for training and testing of traditional machine learning classifiers

In [ ]:
#y_test当中除去o的数目
total = y_test[y_test != 'O']
print(len(total))

TP = 2690
FP = 1810
FN = 3209
TN = total.size - TP - FP - FN
print(TP, FP, FN, TN)

#calculate the accuracy
accuracy = (TP + TN) / (TP + FP + FN + TN)
print(accuracy)

#calculate the precision
precision = TP / (TP + FP)
print(precision)

#calculate the recall
recall = TP / (TP + FN)
print(recall)

#calculate the F1 score
F1 = 2 * precision * recall / (precision + recall)
print(F1)



In [ ]:
#read result.txt
path = 'E:/JHU/课程/datadesign/NLP/machine_learning/stanford-ner-4.2.0/stanford-ner-2020-11-17/test'

data = pd.read_csv(path + '/result_M40_N0_chris2useLC.txt', sep='\t', header=None, names=['Entity', 'Percision', 'Recall', 'F1', 'TP', 'FP', 'FN'])
#Drop the first row
data = data.drop([0])

print(data)
print('')

#Drop the last row
data = data.drop([len(data)])
data = data.reset_index(drop=True)


#convert the data type
data['Percision'] = data['Percision'].astype(float)
data['Recall'] = data['Recall'].astype(float)
data['F1'] = data['F1'].astype(float)
data['TP'] = data['TP'].astype(int)
data['FP'] = data['FP'].astype(int)
data['FN'] = data['FN'].astype(int)
#print(data)

#calculate the micro average
TP = data['TP'].sum()
FP = data['FP'].sum()
FN = data['FN'].sum()
micro_precision = TP / (TP + FP)
micro_recall = TP / (TP + FN)
micro_F1 = 2 * micro_precision * micro_recall / (micro_precision + micro_recall)

#calculate the macro average
macro_precision = data['Percision'].mean()
macro_recall = data['Recall'].mean()
macro_F1 = data['F1'].mean()

#calculate the support
support = data['TP']
#print(support)
#add the support to the dataframe
data['support'] = support
support_proportion = support / support.sum()
#print(support_proportion)

#calculate the weighted average
weighted_precision = (data['Percision'] * support_proportion).sum()
weighted_recall = (data['Recall'] * support_proportion).sum()
weighted_F1 = (data['F1'] * support_proportion).sum()


Evalution = pd.DataFrame(columns=['Entity', 'Percision', 'Recall', 'F1', 'TP', 'FP', 'FN','support'])
Evalution.loc[len(Evalution)] = ['micro-average', micro_precision, micro_recall, micro_F1, TP, FP, FN, support.sum()]
Evalution.loc[len(Evalution)] = ['macro-average', macro_precision, macro_recall, macro_F1, TP, FP, FN, support.sum()]
Evalution.loc[len(Evalution)] = ['weighted-average', weighted_precision, weighted_recall, weighted_F1, TP, FP, FN, support.sum()]


print(data)
print('')
print(Evalution)
